<a href="https://colab.research.google.com/github/abhinav9800/abhinav99/blob/main/image_classification_using_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dataset - https://www.kaggle.com/datasets/pavansanagapati/images-dataset

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d pavansanagapati/images-dataset

 99% 1.00G/1.01G [00:15<00:00, 84.5MB/s]
100% 1.01G/1.01G [00:15<00:00, 68.9MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/images-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
!mkdir /content/data/train
!mkdir /content/data/test

In [5]:
import os
import random
import shutil

In [6]:
!pip install -U efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00


In [7]:
import efficientnet.keras as efn
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.callbacks import EarlyStopping

In [8]:
conv_base = efn.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    classes=7,
    input_shape=(150, 150, 3),
    drop_connect_rate=0.4
)


16804768/16804768 [==============================] - 0s 0us/step


In [9]:
#Splitting image datasets into train and test dataset

data_paths = {
    "car": "/content/data/cars",
    "cat": "/content/data/cats",
    "flower":"/content/data/flowers",
    "horse":"/content/data/horses",
    "human":"/content/data/human",
    "dog":"/content/data/dogs",
    "bike":"/content/data/bike"


}

train_base_folder = "/content/data/train"
test_base_folder = "/content/data/test"

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
random.seed(42)


for category, data_path in data_paths.items():

    train_folder = os.path.join(train_base_folder, category)
    test_folder = os.path.join(test_base_folder, category)

    for folder_path in [train_folder, test_folder]:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)


    imgs_list = [filename for filename in os.listdir(data_path) if os.path.splitext(filename)[-1] in image_extensions]


    random.shuffle(imgs_list)


    train_size = int(len(imgs_list) * 0.85)
    test_size = int(len(imgs_list) * 0.15)


    for i, f in enumerate(imgs_list):
        if i < train_size:
            dest_folder = train_folder
        else:
            dest_folder = test_folder
        shutil.copy(os.path.join(data_path, f), os.path.join(dest_folder, f))

In [10]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

In [11]:
train_dirs = ["/content/data/train/bike",
              "/content/data/train/car",
              "/content/data/train/cat",
              "/content/data/train/flower",
              "/content/data/train/horse",
              "/content/data/train/human",
              "/content/data/train/dog"





]

# Create an ImageDataGenerator for augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Loop over each directory
for data_dir in train_dirs:
    # Get a list of all image filenames in the directory
    image_files = os.listdir(data_dir)


    for image_file in image_files:

        image_path = os.path.join(data_dir, image_file)
        image = load_img(image_path)

        image_array = img_to_array(image)

        image_array = image_array.reshape((1,) + image_array.shape)


        i = 0
        for batch in train_datagen.flow(image_array,
                                         batch_size=1,
                                         save_to_dir=data_dir,
                                         save_prefix='augmented_' + image_file.split('.')[0],
                                         save_format='jpg'):
            i += 1
            if i >= 2:
                break

batch_size= 32
train_generator = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        '/content/data/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 4577 images belonging to 7 classes.
Found 274 images belonging to 7 classes.


In [12]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [13]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/data/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/data/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

Found 4577 files belonging to 7 classes.
Found 274 files belonging to 7 classes.


In [14]:
def process(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [15]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=2)

history = model.fit(train_ds, epochs=10, validation_data=validation_ds,callbacks=[early_stopping_callback])


Epoch 1/10
144/144 [==============================] - 48s 146ms/step - loss: 0.7447 - accuracy: 0.7640 - val_loss: 0.2484 - val_accuracy: 0.9270
Epoch 2/10
144/144 [==============================] - 17s 120ms/step - loss: 0.1441 - accuracy: 0.9596 - val_loss: 0.1167 - val_accuracy: 0.9635
Epoch 3/10
144/144 [==============================] - 18s 121ms/step - loss: 0.0702 - accuracy: 0.9819 - val_loss: 0.0676 - val_accuracy: 0.9781
Epoch 4/10
144/144 [==============================] - 18s 122ms/step - loss: 0.0408 - accuracy: 0.9889 - val_loss: 0.0444 - val_accuracy: 0.9854
Epoch 5/10
144/144 [==============================] - 18s 123ms/step - loss: 0.0241 - accuracy: 0.9948 - val_loss: 0.0374 - val_accuracy: 0.9891
Epoch 6/10
144/144 [==============================] - 18s 122ms/step - loss: 0.0148 - accuracy: 0.9967 - val_loss: 0.0359 - val_accuracy: 0.9891
Epoch 7/10
144/144 [==============================] - 18s 122ms/step - loss: 0.0117 - accuracy: 0.9978 - val_loss: 0.0304 - val_ac